In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools 
import warnings



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
Sales_train

In [ ]:
Sales_train.item_cnt_day.plot()

Считываем входные данные в pandas-формат

In [ ]:
Sales_train.info()

Информация о переменных

In [ ]:
pd.to_datetime(Sales_train.date,dayfirst = True)
Date_time = pd.to_datetime(Sales_train.date,dayfirst = True)
X1 = Sales_train.date_block_num
X2 = Sales_train.shop_id
X3 = Sales_train.item_id
X4 = Sales_train.item_price
Y = Sales_train.item_cnt_day

Выделяем в отдельные переменные дату как Date_time,хар-ки товарра как X1-x4,продажи товара в день как Y

In [ ]:
Date_by_blocks = pd.DataFrame({"Date": Date_time,"Date_blocks": X1})
Date_by_shop_id = pd.DataFrame({"Date": Date_time,"Shop_id": X2})
Date_by_items_id = pd.DataFrame({"Date": Date_time,"Items_id": X3})
Date_by_price = pd.DataFrame({"Date": Date_time,"Price": X4})
Date_by_sales_count = pd.DataFrame({"Date": Date_time,"Sales_count": Y})

Разбиваем данные на временные ряды

In [ ]:
Date_by_blocks.sort_values(by=['Date'])
ys = Date_by_blocks.Date
xs = Date_by_blocks.Date_blocks
plt.figure(figsize=(20,5))
plt.scatter(xs, ys)
plt.grid(True)
plt.show()

In [ ]:
Date_by_shop_id.sort_values(by=['Date'])
ys = Date_by_shop_id.Date
xs = Date_by_shop_id.Shop_id
plt.figure(figsize=(20,5))
plt.scatter(xs, ys)
plt.grid(True)
plt.show()

In [ ]:
Date_by_items_id.sort_values(by=['Date'])
ys = Date_by_items_id.Date
xs = Date_by_items_id.Items_id
plt.figure(figsize=(20,10))
plt.scatter(xs, ys)
plt.grid(True)
plt.show()

In [ ]:
Date_by_price.sort_values(by=['Date'])
xs = Date_by_price.Date
ys = Date_by_price.Price
plt.figure(figsize=(20,10))
plt.scatter(xs, ys)
plt.grid(True)
plt.show()

In [ ]:
Date_by_sales_count.sort_values(by=['Date'])
xs = Date_by_sales_count.Date
ys = Date_by_sales_count.Sales_count
plt.figure(figsize=(20,10))
plt.scatter(xs, ys)
plt.grid(True)
plt.show()

Выводим графики распределения временных рядов

In [ ]:
#a = []
#for i in range(len(Sales_train.groupby("item_id").groups)):
#    filter = Sales_train["item_id"] == i
#    a.append(Sales_train.loc[filter])
a = []
j = Sales_train.groupby("item_id").groups
for i in range(len(Sales_train['item_id'].unique())):
    if(i in j):
        a.append(Sales_train.loc[j[i]])
a[1]

In [ ]:
Sales_train.item_id.value_counts()

In [ ]:
boof1 = Sales_train[(Sales_train.item_id == 20949) & (Sales_train.shop_id == 28)]
boof1.date = pd.to_datetime(boof1.date,dayfirst = True)
boof1 = boof1.sort_values("date")
xs = boof1[(boof1.date > "2014.01.01") & (boof1.date < "2014.12.31")].date
ys = boof1[(boof1.date > "2014.01.01")  & (boof1.date < "2014.12.31")].item_cnt_day
data = boof1[(boof1.date > "2014.01.01") & (boof1.date < "2014.12.31")]
plt.figure(figsize=(20,7))
plt.plot(xs, ys)
plt.grid(True)
plt.show()

In [ ]:
df = pd.DataFrame(np.array(ys), index=xs, columns=['xs'])
df['mean'] = df.rolling(10).mean()
df.plot(figsize=(20,7))

plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
sm.graphics.tsa.plot_acf(ys)
plt.grid(True)
plt.show()

In [ ]:
sm.graphics.tsa.plot_pacf(ys)
plt.grid(True)
plt.show()

In [ ]:
tryw = pd.DataFrame(np.array(ys), index=range(len(xs)))
model = SARIMAX(tryw, order=(2, 0, 2), seasonal_order=(1, 1, 2, 48))
model_fit = model.fit(disp=False)
yhat = model_fit.predict(0, len(ys)+250)
model_fit.aic,len(yhat)

In [ ]:
plt.figure(figsize=(25, 10))
plt.scatter(range(len(boof1[(boof1.date > "2014.01.01")  & (boof1.date < "2020.12.31")].item_cnt_day)), boof1[(boof1.date > "2014.01.01")  & (boof1.date < "2020.12.31")].item_cnt_day, c='b', label='Заданные значения')
plt.plot(yhat, c='r', label='Значения, построенные моделью')
plt.grid(True)
plt.legend(loc='best')
plt.show()

Попробовал разбить список - не ручками же его забивать
Результат так себе был,сейчас вроде получше. Получилось разбить на датафреймы с группировкой по item_id. Определённый товар,кстати,кажется всегда продют в определённом мгазине - что хорошо,не приходится возиться с доопределением

In [ ]:
boof = pd.DataFrame({"time":a[1].date,"y":a[1].item_cnt_day})
boof

In [ ]:
easy_a = []
hard_a = []
for i in range(len(a)):
    if (len(a[i]) >= 1000):
        hard_a.append(i)
    else: 
        easy_a.append(i)
len(easy_a),len(hard_a),hard_a[0]

In [ ]:
a[30]

In [ ]:
xs = a[30].date
ys = a[30].item_cnt_day
plt.figure(figsize=(20,10))
plt.scatter(xs[:50], ys[:50])
plt.grid(True)
plt.show()

In [ ]:
Sales_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
b = []
k = Sales_test.groupby("item_id").groups
for i in range(len(Sales_test['item_id'].unique())):
    if(i in k):
        b.append(Sales_test.loc[k[i]])
b[0]

Работаем с тестоой выборкой

In [ ]:
easy_b = []
for i in range(len(b)):
        easy_b.append(i)
len(easy_b)

In [ ]:
from sklearn.linear_model import LogisticRegression

def easy_predict(df,test):
    model = LogisticRegression()
    boof = pd.DataFrame({"time":df.date,"item_id":df.item_id,"y":df.item_cnt_day})
    boof.time = pd.to_datetime(boof.time,dayfirst = True)
    y = boof.y
    x = boof[["time","item_id"]]
    #model.fit(x,y)
    return df

def hard_predict(df):
    return df

Разбиваем подборку датасетов по размеру - чтобы выбрать способ обработки

In [ ]:
result = []
for i in range(len(a)):
    if (i in easy_b):
        easy_predict(a[i], b[i])
    else:
        easy_predict(a[i], -1)
#for i in range(len(a))
#    if ((i-30) in b):
#        easy_predict(a[i],b.item_id[i])

Забиваем данные в цикл - может долго компилиться. Методы цикла даны выше.